In [1]:
!pip install botorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.2/745.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
import torch
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.gp_regression import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

# isn't this easier?
# https://github.com/pytorch/botorch/discussions/1444
torch.set_default_dtype(torch.double) # double == float64

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Question: How to set the lengthscale for each batch of the kernel?
# (will figure this out)

# https://docs.gpytorch.ai/en/stable/kernels.html#gpytorch.kernels.Kernel
# https://docs.gpytorch.ai/en/stable/_modules/gpytorch/kernels/kernel.html#Kernel

# Question: when maximizing the likelihood of batched GP, then does it maximize the
# sum of all the likelihoods of all the batches, or does it maximize them independently?
# I would prefer the latter, but this tutorial
# https://github.com/cornellius-gp/gpytorch/blob/master/examples/08_Advanced_Usage/Simple_Batch_Mode_GP_Regression.ipynb
# maximizes the sum. But I'm not sure what `botorch.fit.fit_gpytorch_mll` does.

In [ ]:
def calculate_EI_GP(X_hist, y_hist, X, kernel, fit_params=False):
    """Calculate the exact Expected Improvements at `n_eval` points,
    given `N` histories each of length `n_train`.
    Assumes noise-free observations, so gives a fixed noise level of 1e-6

    Args:
        X_hist: History x values, of shape `(N, n_train, d)`
        y_hist: History y values, of shape `(N, n_train)` or `(N, n_train, 1)`
        X: Evaluation x points, of shape `(N, n_eval, d)`
        kernel: the kernel to use
        fit_params: whether to fit parameters by maximizing the marginal log
            likelihood

    Returns:
        A `(N, n_eval)`-dim tensor of Expected Improvement values at the
        given design points `X`.
    """
    # Get y_hist into (N, n_train, 1) shape so we can give to SingleTaskGP
    if y_hist.dim() == 2:
        y_hist = y_hist.unsqueeze(-1)
    elif y_hist.dim() == 3:
        assert y_hist.size(2) == 1
    else:
        raise AssertionError("y_hist dimension must be 2 or 3")

    assert X_hist.dim() == X.dim() == 3
    assert X_hist.size(0) == y_hist.size(0) == X.size(0) # N=N=N
    assert X_hist.size(1) == y_hist.size(1) # n_train=n_train
    assert X_hist.size(2) == X.size(2) # d=d

    y_hist_var = torch.full_like(y_hist, 1e-6)
    model = SingleTaskGP(X_hist, y_hist, y_hist_var, covar_module=kernel)

    if fit_params:
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)

    # best_f has shape (N,)
    best_f = y_hist.squeeze().amax(1) # unsqueezed so need to squeeze again
    EI_object = ExpectedImprovement(model, best_f=best_f, maximize=True)

    # X currently has shape (N, n_eval, d)
    # Make it have shape (b_1, b_2, 1, d) where (b_1, b_2) = (N, n_eval)
    # The added "1" would be the "q" for "q-batch" in general
    X = X.unsqueeze(2)

    # But also need to swap the batch dimensions to align with what it says here
    # https://botorch.org/docs/batching#batched-models
    X = torch.transpose(X, 0, 1) # Now has shape (n_eval, N, 1, d)

    EI_values = EI_object(X) # shape (n_eval, N)

    EI_values = torch.transpose(EI_values, 0, 1) # need to swap again

    return EI_values # shape (N, n_eval)


In [ ]:
x = torch.tensor([0.2, 0.9, 3.1], device=device)
y = torch.tensor([0.1, 0.2, -1.1])
x * y